## Visualize NaNs

In [ ]:
notnull = ~(mosaiks_features["1000"].isnull())*1

f, ax = plt.subplots(1,1)
notnull.cumsum().plot(ax=ax)
ax.plot([0,96167],[0,96167],c="r")
plt.show()

## Load temp files, combine, and save

In [ ]:
import glob

filepaths = sorted(glob.glob(mosaiks_path+"2021_*.parquet.gzip"))

filepaths

temp = pd.read_parquet(
    "/home/jovyan/ds_nudge_up/data/01_preprocessed/mosaiks_features/2021_3km_v4000_L8_60000__0_to_36166.parquet.gzip"
)
temp.index = range(60000, 96167)
temp.to_parquet(
    "/home/jovyan/ds_nudge_up/data/01_preprocessed/mosaiks_features/2021_3km_v4000_L8_60000_to_96166.parquet.gzip", 
    compression="gzip"
)

In [ ]:
filename = '2021_3km_v4000_L8_0_to_96166.parquet.gzip'

l_dfs = []
for filepath in filepaths:
    l_dfs.append(pd.read_parquet(filepath))

mosaiks_features = pd.concat(l_dfs, axis=0)

mosaiks_features.to_parquet(mosaiks_path+filename, compression="gzip")

In [ ]:
# filter points with buffer

states = gpd.read_file(f"{DATA_ROOT}/00_raw/SHRUG/geometries_shrug-v1.5.samosa-open-polygons-shp/state.shp")
states['zero_column'] = 0
country = states.dissolve(by='zero_column')
BUFFER_DISTANCE = 0.005
points_gdf = filter_points_with_buffer(points_gdf, country, BUFFER_DISTANCE)

## Load MOSAIKS features

In [ ]:
# Load custom-made mosaiks features
# data_label = "3km_v1024_L8"
# filename = 'mosaiks_2013_L8_0.015b_all_points.parquet.gzip'

# data_label = "1km_v1024_L8"
# filename = 'mosaiks_2013_L8_0.005b_all_points.parquet.gzip'

# lat_name, lon_name = "lat", "lon"

# Load data
# mosaiks_features = pd.read_parquet(mosaiks_path+filename)

In [ ]:
# # OR using old data downloaded from the online API
# data_label = "old_v4000"
# filename = "mosaiks_downloaded_old_features_rounded.csv"
# lat_name, lon_name = "Lat", "Lon"

# # Load data
# mosaiks_features = pd.read_csv(mosaiks_path+filename)

In [ ]:
mosaiks_features_gdf = load_mosaiks_data("INDIA_SHRUG_Mosaiks_features.csv")
# mosaiks_features_gdf.head()

In [ ]:
mosaiks_features_gdf.shape

## Load preprocessed SHRUG keys with shapes

In [ ]:
shrug_key_geoms = load_gdf("01_preprocessed/SHRUG/shrug_pc11r_key_with_shapes", "shrug_pc11r_key_with_shapes.shp")
# shrug_key_geoms.head()

## Use SHRUG shapes to add `shrid`s to MOSAIKS features

In [ ]:
mosaiks_features_gdf = add_shrid_to_mosaiks(mosaiks_features_gdf, shrug_key_geoms)

## Import SHRUG SECC (target)

In [ ]:
file_path = Path.cwd().parent / "data" / "00_raw" / "SHRUG" / "shrug-v1.5.samosa-secc-csv" / "shrug_secc.csv"
shrug_secc = pd.read_csv(file_path)
# shrug_secc.head()

In [ ]:
shrug_secc.shape

## Match target to features using `shrid`s

In [ ]:
df = merge_mosaiks_and_secc(mosaiks_features_gdf, shrug_secc)
# df.head()

In [ ]:
df.shape